# Bilbiotecas

In [ ]:
import requests

Faz uma solicitacao GET ao servidor, para o **download dos conteudos HTML da pagina** solicitada


In [ ]:
#import utils as tools 

Funções criadas pelo desenvolvedor **(eu, Carlos Neto)** para agilizar e otimizar o codigo (contem **calculo de datas tratameto de string)**

In [ ]:
from time import sleep

Função _time_ responsável por **pausas de segundos no código**, para evitar DoS de flood de requisao ao site, preservando o funcionamento do memso

In [ ]:
from bs4 import BeautifulSoup 

Biblioteca para o **parseameto do codigo html**

In [ ]:
import pandas as pd 

Biblioteca para análise de dados, possibilita no caso, **salvar o Json** preservando a integridade dos dados

# Escopo da Classe ScrapEmpregaSjc() 
>Classe para realizar scrap do site Emprega Sao Jose

In [ ]:
class ScrapEmpregaSjc():
    def __init__(self):
        self.urlBase = "https://www.empregasaojosecampos.com"
        self.jobs = {}

### Função para instanciar objetos "soup", para o reaproveitamento de codigo

In [ ]:
    def make_soup(self, url):
        page = requests.get(url)
        return BeautifulSoup(page.content, 'html.parser')

* <big>**page = requests.get(url)**</big> : Obtem por meio da biblioteca requests o código fonte da página recebendo como parâmetro a URL do site
* <big>**return BeautifulSoup(page.content, 'html.parser')**</big> : Retorna uma Instância da biblioteca BeautifulSoup

### Escopo do método *empregos_api(self)*
>Função responsável por fazer o scrap das páginas

In [ ]:
    def empregos_api(self, url):
        soup = self.make_soup(url)
        lista_vagas = soup.findAll("div", class_ = "blog-post")

* <big>**soup = self.make_soup(url)**</big> Instancia um objeto "soup" com a url da pagina
* <big>**lista_vagas = soup.findAll("div", class_ = "blog-post")**</big> Cria uma lista com o codigo que detem os enunciados da pagina

In [ ]:
        k = 0
        while( k <= len(lista_vagas[:-1]) ) :
            headerVaga = lista_vagas[k].find("span", class_ = "blog-post-title") 
            dadosVaga = lista_vagas[k].find("div", class_ = "metadata") 
            caracteristicas = dadosVaga.findAll("span") 
            nomeVaga = tools.tratar_nome_vaga(self, headerVaga.getText()) 

* <big>**k = 0**</big> Contador para navegar entre os indices dos enunciados da pagina
* <big>**while(k <= len(lista_vagas[:-1]) ) :**</big>
>* <big>**headerVaga = lista_vagas[k].find("span", class_ = "blog-post-title")**</big> Filtra apenas os dados respectivos ao titulo da vaga
>* <big>**dadosVaga = lista_vagas[k].find("div", class_ = "metadata")**</big> Filtra os dados referente ao corpo do enunciado da vaga
>* <big>**caracteristicas = dadosVaga.findAll("span")**</big> Faz outro filtro para facilitar a obtencao dos dados que interessam
>* <big>**nomeVaga = tools.tratar_nome_vaga(self, headerVaga.getText())**</big> Esta variavel passara por um tratamento", preservando a consistencia do dado


In [ ]:
            self.jobs[nomeVaga] = {
                    'local': caracteristicas[2].getText() + ', ' + caracteristicas[3].getText() + ' - ' + caracteristicas[4].getText(),
                    'data_vaga':  dadosVaga.find("time", class_ = "value-title").getText().strip(),
                    'salario': caracteristicas[5].getText(),
                    'como_concorrer': headerVaga.find("a").get("href"),
                    'mais_info':
                        'Nivel Academico: ' + caracteristicas[7].getText()+' - '
                        +'Status Vaga:' + caracteristicas[8].getText()
                }

<big>**Estrutura do .Json**</big>
* funcao (Sera o índice da vaga, pelo qual inserimos o nome da vaga)

>* local:
>>Contém informações como Estado/Cidade 
ex: (Jacareí, SP)

>* data_vaga:
>>Detém informações referente ao dia que foi aberto ou divulgado a vaga
ex: (23-01-2018)

>* salario:
>>Contém o salario(evitando casas decimais), caso o enunciado não incluir tal informação, colocar "combinar", ex: (1500) para mil quinhentos reais, e, ('combinar') para ausência de informação 

>* como_concorrer :
>>Links, e-mails da empresa

>* mais_info
>>Campo aberto para o dados geral, 
ex: (requisitos acadêmicos, vaga efetiva ou temporaria)

In [ ]:
            print (self.jobs[nomeVaga])
            pd.DataFrame(self.jobs).to_json('empregaOutput.json')
            k += 1
            sleep(3)

* <big>**print (self.jobs[nomeVaga])**</big> Efeitos de visualizacao somente... (ver no console o progresso da raspagem)
* <big>**pd.DataFrame(self.jobs).to_json('empregaOutput.json')**</big> Salvando o resultado em .json
* <big>**k += 1**</big> Incrementa o contador
* <big>**sleep(3)**</big> Pausa de 3 segundos a cada fim de Scrap, para evitar possível DoS de flood de requisão ao site, preservando o funcionamento do mesmo

### Escopo do método *pes_ultimas_vagas(self)*
>Função responsável por fazer o scrap das páginas
```
PADRÃO DE ESTRUTURA DE URL DO SITE:
  
> Pagina 1(home): 
https://www.empregasaojosecampos.com/
  
> Pagina 2:
https://www.empregasaojosecampos.com/page/2
```

O que muda na estrutura é o empregasaojose<big>**/page/NUMERO-DA-PAGINA**</big>

In [ ]:
        pageNumber = 1 
        qtd_pagina = 5

* <big>**pageNumber = 1**</big> É o responsável por ser o índice que serve como indicador da página que será feito o scrap 
* <big>**qtd_pagina = 5**</big> Quantidade de pagina que será executado o scrap

In [ ]:
        while (pageNumber <= qtd_pagina):  
            if pageNumber == 1:
                self.empregos_api(self.urlBase)
            else:
                self.empregos_api(self.urlBase + "/page/" + str(pageNumber) )
            pageNumber+=1

* <big>**while (pageNumber <= qtd_pagina):** </big> Laço para percorrer as páginas

>* <big>**if pageNumber == 1:**</big> Caso for página "1", Não necessário a modificação da url
>
>>*            <big>**self.empregos_api(self.urlBase)**</big> Executa o scrap da "pagina 1"
>
>*            <big>**else:**</big>
>
>>*                <big>**self.empregos_api(self.urlBase + "/page/" + str(pageNumber) )**</big> Faz o scrap modificando a url
>
>*            <big>**pageNumber+=1**</big> Avança o número da página

### Escopo do método *pes_vagas_nome(self)*
>Função responsável por fazer o scrap das páginas

PADRAO DE URL
Pesquisa por: ajudante e operador 

* ajudante pagina 1:
  https://www.empregasaojosecampos.com/?s=ajudante
* ajudante pagina 2:
  https://www.empregasaojosecampos.com/page/2?s=ajudante
            

* operador pagina 1: 
  https://www.empregasaojosecampos.com/?s=operador

* operador pagina 2:
https://www.empregasaojosecampos.com/page/2?s=operador

O que muda na estrutura é:
* Para primeira página:   
    url-base+ <big>/?s=*nome_da_funcao*/</big>


* Para Segunda e demais páginas:
    url-base+ <big>/**NumeroDaPagina**?s=*nome_da_funcao*/</big>

In [1]:
    def pes_vaga_nome(self, vaga):
        pageNumber = 1 
        qtd_pagina = 5 

        while (pageNumber <= qtd_pagina): 
            if pageNumber == 1: 
                self.empregos_api(self.urlBase + "/?s=" + vaga)
            else:
                self.empregos_api(self.urlBase + "/page/" + str(pageNumber) + "?s=" + vaga)
            pageNumber+=1

* ```pageNumber = 1``` Contador de pagina
* ```qtd_pagina = 5``` Quantidade de paginas que sera feito o scrap

* ```while (pageNumber <= qtd_pagina):``` Laço para percoperrrer as paginas
>
>```if pageNumber == 1:``` Caso for pagina "1", tera modificacoes da url diferente com as demais
>>```self.empregos_api(self.urlBase + "/?s=" + vaga)``` Executa o scrap da "pagina 1"
>
>```else:```
>>```self.empregos_api(self.urlBase + "/page/" + str(pageNumber) + "?s=" + vaga)``` Faz o scrap modificando a url
>
>```pageNumber+=1``` Avança o número da página

# Escopo da *main()* 
>Metódo de start 

In [2]:
if __name__ == '__main__':
    categorias = ['Ajudante','Analista','Assistente','Atendente','Auxiliar','Balconista','Caixa','Comercial','Coordenador','Copeira','Cozinheiro','Departamento Pessoal','Empilhadeira','Eletricista','Encarregado','Enfermagem','Estoquista','Farmacêutico','Jovem Aprendiz','Mecânico','Manutenção','Motorista','Nutricionista','Produção','Operador','Pedreiro','Secretária','Promotor','Recepcionista','RHServiços Gerais','Telemarketing','Técnico','Vendedor']